In [1]:
import torch
import dqc

In [2]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p

In [3]:
def DQC_calculation_tensors(system, a, p):
    atomzs, atomposs = dqc.parse_moldesc(system)
    mol = dqc.Mol((atomzs, atomposs), basis="6-31G")
    
    myxc = MyLDAX(a, p)

    qc = dqc.KS(mol, xc=myxc).run()
    ene = qc.energy()  # calculate the energy
    return ene

In [4]:
def DQC_calculation_values(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    return DQC_calculation_tensors(system, a, p)

In [17]:
def finite_difference(system, a_value, p_value, coeff):
    y = DQC_calculation_values(system, a_value, p_value)
    da = a_value * coeff
    dp = p_value * coeff
    y_plus_dy_a = DQC_calculation_values(system, a_value+da, p_value)
    y_plus_dy_p = DQC_calculation_values(system, a_value, p_value+dp)
    dy_wrt_da = (y_plus_dy_a - y) / da
    dy_wrt_dp = (y_plus_dy_p - y) / dp
    return (dy_wrt_da, dy_wrt_dp)

In [8]:
def autograd(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    y = DQC_calculation_tensors(system, a, p)
    return torch.autograd.grad(y, (a, p))

In [26]:
h2_system = "H 0 0 0; H 1.4 0 0"
print("autograd results: ", autograd(h2_system, 1, 2))
print("finite difference results: ", finite_difference(h2_system, 1, 2, 0.0001))

autograd results:  (tensor(0.0885, dtype=torch.float64), tensor(-0.2367, dtype=torch.float64))
finite difference results:  (tensor(0.0885, dtype=torch.float64, grad_fn=<DivBackward0>), tensor(-0.2367, dtype=torch.float64, grad_fn=<DivBackward0>))


In [27]:
print("autograd results: ", autograd(h2_system, 44, 11))
print("finite difference results: ", finite_difference(h2_system, 44, 11, 0.0001))

autograd results:  (tensor(1.5659e-08, dtype=torch.float64), tensor(-1.0437e-06, dtype=torch.float64))
finite difference results:  (tensor(1.5659e-08, dtype=torch.float64, grad_fn=<DivBackward0>), tensor(-1.0428e-06, dtype=torch.float64, grad_fn=<DivBackward0>))


In [24]:
n2_system = "N 0 0 0; N 2.07 0 0"
print("autograd results: ", autograd(n2_system, 1, 2))
print("finite difference results: ", finite_difference(n2_system, 1, 2, 0.01))

autograd results:  (tensor(14.7474, dtype=torch.float64), tensor(25.9162, dtype=torch.float64))
finite difference results:  (tensor(15.7496, dtype=torch.float64, grad_fn=<DivBackward0>), tensor(25.2596, dtype=torch.float64, grad_fn=<DivBackward0>))
